In [1]:
import numpy as np
import pandas as pd
from botorch.test_functions.synthetic import Ackley, AckleyMixed, Labs, Griewank
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm

import sys
sys.path.append('../')
from roelfes_emulator import RoelfesEmulator
from vae import Autoencoder, WeightedAutoencoder
from bayesian_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt


In [2]:
def generate_random_inputs(bounds, q, discrete_inds=None):
    lower, upper = bounds  
    generated_samples = lower + (upper - lower) * torch.rand((q, bounds.shape[1]))
    if discrete_inds is not None:
        generated_samples[:, discrete_inds] = torch.round(generated_samples[:, discrete_inds])
    return generated_samples

In [3]:
def bayes_opt_roelfes_coef(objs, q, q_sampling_method="Monte Carlo"):
    lin_y_names= ['y1','y2']
    dec_exp_y_names = ['y3','y4','y5','y6']
    all_names = lin_y_names + dec_exp_y_names

    lin_df = objs.drop(dec_exp_y_names,axis=1)
    dec_exp_df = objs.drop(lin_y_names, axis=1)

    q1 = q // 2
    q2 = q - q1
    if q == 1:
        q1 = 1

    lin_bo_model = BayesianOptimization().fit(lin_df, y=lin_y_names)
    c1 = lin_bo_model.candidates(q1, export_df=True, q_sampling_method=q_sampling_method)
    dec_exp_bo_model = BayesianOptimization().fit(dec_exp_df, y=dec_exp_y_names, 
                                                optim_direc=['max','max', 'min','max'])
    c2 = dec_exp_bo_model.candidates(q2, export_df=True,q_sampling_method=q_sampling_method)
    cand = pd.concat((c1, c2)).fillna(0)
    pred = cand[all_names].to_numpy()
    cand = cand.drop(all_names, axis=1).to_numpy()
    if q == 1:
        choice = np.random.randint(0,2)
        return cand[choice,:], pred[choice,:]
        
    return cand, pred

In [4]:
# Example definitions (ensure you run these first)
starting_samples_n = 96
cv_amount = 2
q_list = [1, 96]
sampling_strategies = ["Monte Carlo", "Believer"]
test_component_choice_list = [2, 16, 32]
dim_red_list = [PCA, Autoencoder, WeightedAutoencoder, None]  # Replace with actual class references

# Define your function list and objective directions
function_list = [
    ("Ackley", Ackley(dim=50)),
    ("AckleyMixed", AckleyMixed(dim=53)),
    ("Labs", Labs(dim=50)),
    ("Griewank", Griewank(dim=50)),
    ("RoelfesCoef", RoelfesEmulator(dim=50, objective="coef", use_torch=True)),
    ("RoelfesMaxYield", RoelfesEmulator(dim=50, objective="max yield", use_torch=True)),
    ("RoelfesNormGrad", RoelfesEmulator(dim=50, objective="normalized yield gradient", use_torch=True)),
]

optim_direc_map = {
    "Ackley": ["min"],
    "AckleyMixed": ["min"],
    "Labs": ["min"],
    "Griewank": ["min"],
    "RoelfesCoef": ["max", "max", "max", "max", "min", "max"],
    "RoelfesMaxYield": ["max"],
    "RoelfesNormGrad": ["max"],
}


In [5]:
from joblib import Parallel, delayed
import itertools


def run_experiment(function_name, function, cv_iter, dr_method, n_components, q, q_method):
    optim_direc = optim_direc_map[function_name]
    weights = np.array([1 if d == "max" else -1 for d in optim_direc])

    # Generate input data
    X = generate_random_inputs(function.bounds, starting_samples_n, function.discrete_inds)
    y = function(X)
    x_np, y_np = X.numpy(), y.numpy()
    x_np_backup, y_np_backup = x_np.copy(), y_np.copy()

    dr_name = dr_method.__name__ if dr_method else "None"
    q_key = f"q{q}_{q_method or 'None'}"

    best_vals = []
    mae_pred_vals = []
    recon_mae_vals = []

    for bo_iter in range(5):
        x_np, y_np = x_np_backup.copy(), y_np_backup.copy()

        if dr_method:
            dr_model = dr_method(n_components=n_components)
            if dr_name == "WeightedAutoencoder":
                x_np_dr = dr_model.fit_transform(x_np, y_np, optim_direc=optim_direc)
            else:
                x_np_dr = dr_model.fit_transform(x_np)
            x_np_recon = dr_model.inverse_transform(x_np_dr)
            recon_mae_vals.append(mean_absolute_error(x_np_recon, x_np))
        else:
            x_np_dr = x_np
            recon_mae_vals.append(mean_absolute_error(x_np_dr, x_np))

        x_cols = [f"x{i+1}" for i in range(x_np_dr.shape[1])]
        if len(optim_direc) > 1:
            y_cols = [f"y{i+1}" for i in range(y_np.shape[1])]
        else:
            y_cols = ['y1']
            y_np = y_np.reshape(len(y_np), 1)

        df = pd.DataFrame(np.hstack([x_np_dr, y_np]), columns=x_cols + y_cols)

        if isinstance(function, RoelfesEmulator) and function.objective == "coef":
            cand, pred = bayes_opt_roelfes_coef(df, q, q_method)
            pred = pred.reshape(1, len(pred))
        else:
            model = BayesianOptimization().fit(df, y_cols, optim_direc=optim_direc)
            cand, pred = model.candidates(q, q_sampling_method=q_method)

        recon_cand = dr_model.inverse_transform(cand) if dr_method else cand
        rc_tensor = torch.tensor(recon_cand)
        recon_cand = torch.max(torch.min(rc_tensor, function.bounds[1].unsqueeze(0)),
                               function.bounds[0].unsqueeze(0))
        if function.discrete_inds is not None:
            recon_cand[:, function.discrete_inds] = torch.round(recon_cand[:, function.discrete_inds])
        new_y = function(recon_cand).numpy().reshape(q, 1)
        recon_cand = rc_tensor.numpy()

        scalar_y = np.dot(new_y, weights.T) if len(weights) > 1 else new_y.flatten() * weights[0]
        best_vals.append(np.max(scalar_y))

        pred_np = np.array(pred)
        mae_pred_vals.append(mean_absolute_error(pred_np, new_y))

        x_np = np.vstack([x_np, recon_cand])
        y_np = np.vstack([y_np, new_y])

    return (function_name, dr_name, n_components, q_key, best_vals, mae_pred_vals, recon_mae_vals, x_np)


# Generate parameter combinations
tasks = []
for function_name, function in function_list:
    for cv_iter in range(cv_amount):
        for dr_method in dim_red_list:
            for n_components in test_component_choice_list:
                for q in q_list:
                    sampling_strats = sampling_strategies if q > 1 else [None]
                    for q_method in sampling_strats:
                        tasks.append((function_name, function, cv_iter, dr_method, n_components, q, q_method))

# Run in parallel (adjust n_jobs based on available CPUs)
results_list = Parallel(n_jobs=-1, verbose=10)(
    delayed(run_experiment)(*args) for args in tasks
)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 16.2min


KeyboardInterrupt: 

In [ ]:
# train_x = train_x.to_numpy(dtype=np.float64).reshape(-1,np.shape(train_x)[1])
# train_y = train_y.to_numpy(dtype=np.float64).reshape(-1,np.shape(train_y)[1])
#for bayesian_opt.py

In [ ]:
results = {}
data = {}

In [ ]:
for function_name, dr_name, n_components, q_key, best_vals, mae_vals, recon_mae, final_x in results_list:
    r = results.setdefault(function_name, {}).setdefault(dr_name, {}).setdefault(n_components, {}).setdefault(q_key, {
        "x": [],
        "best_vals_all_folds": [],
        "mae_vals_all_folds": [],
        "recon_mae_all_folds": []
    })
    d = data.setdefault(function_name, {}).setdefault(dr_name, {}).setdefault(n_components, {}).setdefault(q_key, {})

    r["x"].append(final_x)
    r["best_vals_all_folds"].append(best_vals)
    r["mae_vals_all_folds"].append(mae_vals)
    r["recon_mae_all_folds"].append(recon_mae)

# Final aggregation same as before
for fname in results:
    for dr in results[fname]:
        for comp in results[fname][dr]:
            for q_key in results[fname][dr][comp]:
                data[fname][dr][comp][q_key]["best_vals_avg"] = np.mean(
                    results[fname][dr][comp][q_key]["best_vals_all_folds"], axis=0)
                data[fname][dr][comp][q_key]["mae_vals_avg"] = np.mean(
                    results[fname][dr][comp][q_key]["mae_vals_all_folds"], axis=0)
                data[fname][dr][comp][q_key]["recon_mae_avg"] = np.mean(
                    results[fname][dr][comp][q_key]["recon_mae_all_folds"], axis=0)
